In [0]:
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

#additional imports
from pyspark.sql.functions import when 
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors, Vector
from pyspark.ml.functions import vector_to_array

In [0]:
spark = SparkSession(sc)
sqlc=SQLContext(sc)
print(sc.getConf().getAll())

[('spark.files.useFetchCache', 'false'), ('spark.databricks.preemption.enabled', 'true'), ('spark.hadoop.fs.abfs.impl', 'shaded.databricks.v20180920_b33d810.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem'), ('spark.driver.tempDirectory', '/local_disk0/tmp'), ('spark.hadoop.fs.adl.impl.disable.cache', 'true'), ('spark.hadoop.parquet.block.size.row.check.max', '10'), ('spark.hadoop.fs.s3a.connection.maximum', '200'), ('spark.databricks.clusterUsageTags.numPerClusterInitScriptsV2', '0'), ('spark.hadoop.fs.s3a.fast.upload.active.blocks', '32'), ('spark.databricks.clusterUsageTags.driverInstanceId', 'i-0a28c1813dcb28818'), ('spark.shuffle.reduceLocality.enabled', 'false'), ('spark.sql.streaming.checkpointFileManagerClass', 'com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager'), ('spark.databricks.service.dbutils.repl.backend', 'com.databricks.dbconnect.ReplDBUtils'), ('spark.databricks.clusterUsageTags.driverNodeType', 'dev-tier-node'), ('spark.hadoop.spark.sql.sources.outputCommitterClass', 'com.databricks.backend.daemon.data.client.MapReduceDirectOutputCommitter'), ('spark.hadoop.fs.AbstractFileSystem.gs.impl', 'shaded.databricks.V2_1_4.com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS'), ('spark.databricks.clusterUsageTags.instanceBootstrapType', 'ssh'), ('spark.streaming.driver.writeAheadLog.allowBatching', 'true'), ('spark.databricks.clusterSource', 'UI'), ('spark.databricks.clusterUsageTags.sparkVersion', '7.3.x-scala2.12'), ('spark.hadoop.hive.server2.transport.mode', 'http'), ('spark.hadoop.hive.server2.thrift.http.cookie.auth.enabled', 'false'), ('spark.executor.memory', '8278m'), ('spark.databricks.driverNodeTypeId', 'dev-tier-node'), ('spark.sql.parquet.compression.codec', 'snappy'), ('spark.databricks.clusterUsageTags.hailEnabled', 'false'), ('spark.databricks.clusterUsageTags.clusterLogDeliveryEnabled', 'false'), ('spark.databricks.clusterUsageTags.containerType', 'LXC'), ('spark.eventLog.enabled', 'false'), ('spark.databricks.clusterUsageTags.isIMv2Enabled', 'false'), ('spark.hadoop.databricks.s3.create.deleteUnnecessaryFakeDirectories', 'false'), ('spark.hadoop.fs.wasb.impl', 'shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem'), ('spark.executor.tempDirectory', '/local_disk0/tmp'), ('spark.databricks.sparkContextId', '435265476866356109'), ('spark.databricks.workerNodeTypeId', 'dev-tier-node'), ('spark.databricks.clusterUsageTags.driverContainerId', 'e6fd0bb167084c629d9baebdd509415a'), ('spark.hadoop.mapred.output.committer.class', 'com.databricks.backend.daemon.data.client.DirectOutputCommitter'), ('spark.hadoop.hive.server2.thrift.http.port', '10000'), ('spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version', '2'), ('spark.sql.allowMultipleContexts', 'false'), ('spark.databricks.clusterUsageTags.clusterEbsVolumeSize', '0'), ('spark.hadoop.spark.driverproxy.customHeadersToProperties', 'X-Databricks-SqlGateway-SessionId:X-Databricks-SqlGateway-SessionId,X-Databricks-User-Token:spark.databricks.token,X-Databricks-Api-Url:spark.databricks.api.url,X-Databricks-ADLS-Gen1-Token:spark.databricks.adls.gen1.token,X-Databricks-ADLS-Gen2-Token:spark.databricks.adls.gen2.token,X-Databricks-AWS-Credentials:spark.databricks.aws.creds,X-Databricks-User-Id:spark.databricks.user.id,X-Databricks-User-Name:spark.databricks.user.name'), ('spark.home', '/databricks/spark'), ('spark.databricks.clusterUsageTags.clusterTargetWorkers', '0'), ('spark.sql.warehouse.dir', '/user/hive/warehouse'), ('spark.hadoop.hive.server2.idle.operation.timeout', '7200000'), ('spark.task.reaper.enabled', 'true'), ('spark.databricks.passthrough.s3a.tokenProviderClassName', 'com.databricks.backend.daemon.driver.aws.AwsCredentialContextTokenProvider'), ('spark.storage.memoryFraction', '0.5'), ('spark.databricks.session.share', 'false'), ('spark.databricks.clusterUsageTags.clusterResourceClass', 'default'), ('spark.databricks.clusterUsageTags.clusterFirstOnDemand', '0'), ('spark.databricks.clusterUsageTags.clusterName', 'mie

In [0]:
#import table to dataframe
train_df = spark.read.csv('/FileStore/tables/train_data.csv', header='true')
train_df.show()
train_df.cache()

+--------+-------+-------------------+----------+----+------+---+---+------+--------+------+--------------+--------+---------+---------+----------+-------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+
 id| spkid| full_name| pdes|name|prefix|neo|pha| H|diameter|albedo|diameter_sigma|orbit_id| epoch|epoch_mjd| epoch_cal|equinox| e| a| q| i| om| w| ma| ad| n| tp| tp_cal| per| per_y| moid| moid_ld| sigma_e| sigma_a| sigma_q| sigma_i| sigma_om| sigma_w| sigma_ma| sigma_ad| sigma_n| sigma_tp| sigma_per|class| rms|
+--------+-------+-------------------+----------+----+------+---+---+------+--------+------+--------------+--------+---------+---------+----------+-------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+
bK13N34R|3921151| (2013 NR34)| 2013 NR34|null| null| N| N| 19.1| null| null| null| 1|2459000.5| 59000|20200531.0| J2000| 0.2395140902948033| 2.467962162977906|1.8768504506302568|4.2847086382780155| 187.4435648528415| 162.4326374957067|252.60662934088057|3.0590738753255535| 0.2542121643500829| 2459422.955671756|20210727.455671802| 1416.139943265003|3.8771798583572994| 0.871481| 339.15426077|1.663799999999999...|2.826199999999999...|3.921100000000000...|2.170800000000000...| 0.00012328| 0.0013828|0.000782799999999...|3.503099999999999...| 4.3666e-08| 0.0030714| 0.00024325| MBA| 0.54462|
bK15P50H|3725394| (2015 PH50)| 2015 PH50|null| null| N| N|19.699| null| null| null| JPL 3|2457245.5| 57245|20150811.0| J2000| 0.2369724826300891| 2.365728138761921| 1.805115668491649|3.2149011772728504| 58.35429368550654| 290.4174436760314| 345.8561643481903| 2.926340609032194| 0.2708674508386901| 2457297.716815302| 20151002.2168153| 1329.063344027966| 3.63877712259539| 0.801128| 311.77498376| 0.0033089|0.015305000000000001| 0.0056419|0.024416999999999998| 0.57431| 4.3121| 2.1855| 0.018932| 0.0026285| 8.4517| 12.897| MBA| 0.49686|
a0443673|2443673|443673 (2015 HD155)| 443673|null| null| N| N| 16.4| null| null| null| 9|2459000.5| 59000|20200531.0| J2000| 0.2503428686832137| 2.715705981368436| 2.0358483554925| 11.03984398082382|236.99168698795668| 51.7185707883975|349.60064228534617|3.3955636072443722| 0.2202317605851152|2459047.7200634778| 20200717.2200635| 1634.641611380422|4.4754048223967695| 1.04054| 404.9469518|4.520900000000000...| 4.5365e-08| 1.28e-07| 9.3045e-06|3.971300000000000...|4.583399999999999...| 2.832e-05| 5.6721e-08|5.518300000000001...| 0.00012943| 4.0959e-05| MBA| 0.5393|
bK16U95M|3762976| (2016 UM95)| 2016 UM95|null| null| N| N|19.048| null| null| null| 4|2459000.5| 59000|20200531.0| J2000| 0.1392587161510286|2.5278053684836403| 2.17578643818893| 6.0638148322223| 22.79156041768309|350.82965423869337|335.73666891094086|2.8798242987783484| 0.2452384858901295|2459099.4376973235|20200906.937697303| 1467.958826663468| 4.01905222905809| 1.1768|457.97525599999994|0.001169299999999...|0.004072800000000...| 0.0064406|0.050769999999999996| 0.050185| 0.72613| 0.29743| 0.00464| 0.0005927| 1.4209| 3.5478| MBA| 0.51993|
bK08CM9K|3961319| (2008 CK229)|2008 CK22

In [0]:
#neo and pha are both catagorical, turn Y to 1, N to 0
distinctDF = train_df.select('neo').distinct()
distinctDF.show()
distinctDF = train_df.select('pha').distinct()
distinctDF.show()
countDF = (train_df.groupBy('neo').count())
countDF.show()
countDF = (train_df.groupBy('pha').count())
countDF.show()

+----+
 neo|
+----+
 Y|
 N|
null|
+----+

+---+
pha|
+---+
 Y|
 N|
+---+

+----+------+
 neo| count|
+----+------+
 Y| 18369|
 N|732511|
null| 2|
+----+------+

+---+------+
pha| count|
+---+------+
 Y| 1653|
 N|749229|
+---+------+

In [0]:
df1 = train_df.withColumn('neo', when(train_df.neo.isNull(),lit('N')).otherwise(train_df.neo))
countDF = (df1.groupBy('neo').count())
countDF.show()

+---+------+
neo| count|
+---+------+
 Y| 18369|
 N|732513|
+---+------+

In [0]:
df1 = df1.withColumn('neo', regexp_replace('neo', 'N', '0'))
df1 = df1.withColumn('neo', regexp_replace('neo', 'Y', '1'))
df1 = df1.withColumn('pha', regexp_replace('pha', 'N', '0'))
df1 = df1.withColumn('pha', regexp_replace('pha', 'Y', '1'))

In [0]:
distinctDF = df1.select('neo').distinct()
distinctDF.show()
distinctDF = df1.select('pha').distinct()
distinctDF.show()
countDF = (df1.groupBy('neo').count())
countDF.show()
countDF = (df1.groupBy('pha').count())
countDF.show()

+---+
neo|
+---+
 0|
 1|
+---+

+---+
pha|
+---+
 0|
 1|
+---+

+---+------+
neo| count|
+---+------+
 0|732513|
 1| 18369|
+---+------+

+---+------+
pha| count|
+---+------+
 0|749229|
 1| 1653|
+---+------+

In [0]:
#class feature has 13 catagories, turn into index and onehot
distinctDF = df1.select('class').distinct()
distinctDF.show()
countDF = (df1.groupBy('class').count())
countDF.show()

+-----+
class|
+-----+
 OMB|
 IEO|
 ATE|
 TJN|
 APO|
 AST|
 TNO|
 IMB|
 MCA|
 AMO|
 CEN|
 MBA|
 HYA|
+-----+

+-----+------+
class| count|
+-----+------+
 OMB| 22261|
 IEO| 20|
 ATE| 1380|
 TJN| 6531|
 APO| 10183|
 AST| 61|
 TNO| 2728|
 IMB| 15869|
 MCA| 14778|
 AMO| 6786|
 CEN| 402|
 MBA|669881|
 HYA| 2|
+-----+------+

In [0]:
indexer = StringIndexer(inputCol = 'class', outputCol = 'class_index')
model = indexer.fit(df1)
indexed = model.transform(df1)
distinctDF = indexed.select('class','class_index').distinct()
distinctDF.show()
countDF = (indexed.groupBy('class','class_index').count())
countDF.show()

+-----+-----------+
class|class_index|
+-----+-----------+
 MCA| 3.0|
 CEN| 9.0|
 AMO| 5.0|
 IEO| 11.0|
 AST| 10.0|
 TNO| 7.0|
 APO| 4.0|
 TJN| 6.0|
 MBA| 0.0|
 OMB| 1.0|
 ATE| 8.0|
 IMB| 2.0|
 HYA| 12.0|
+-----+-----------+

+-----+-----------+------+
class|class_index| count|
+-----+-----------+------+
 MCA| 3.0| 14778|
 CEN| 9.0| 402|
 AMO| 5.0| 6786|
 IEO| 11.0| 20|
 AST| 10.0| 61|
 TNO| 7.0| 2728|
 APO| 4.0| 10183|
 TJN| 6.0| 6531|
 MBA| 0.0|669881|
 OMB| 1.0| 22261|
 ATE| 8.0| 1380|
 IMB| 2.0| 15869|
 HYA| 12.0| 2|
+-----+-----------+------+

In [0]:
encoder = OneHotEncoder(inputCol = 'class_index', outputCol = 'class_binary')
model = encoder.fit(indexed)
encoded = model.transform(indexed)
distinctDF = encoded.select('class','class_index','class_binary').distinct()
distinctDF.show()
countDF = (encoded.groupBy('class','class_index','class_binary').count())
countDF.show()

+-----+-----------+---------------+
class|class_index| class_binary|
+-----+-----------+---------------+
 APO| 4.0| (12,[4],[1.0])|
 TNO| 7.0| (12,[7],[1.0])|
 AST| 10.0|(12,[10],[1.0])|
 ATE| 8.0| (12,[8],[1.0])|
 IEO| 11.0|(12,[11],[1.0])|
 IMB| 2.0| (12,[2],[1.0])|
 CEN| 9.0| (12,[9],[1.0])|
 MBA| 0.0| (12,[0],[1.0])|
 TJN| 6.0| (12,[6],[1.0])|
 AMO| 5.0| (12,[5],[1.0])|
 OMB| 1.0| (12,[1],[1.0])|
 MCA| 3.0| (12,[3],[1.0])|
 HYA| 12.0| (12,[],[])|
+-----+-----------+---------------+

+-----+-----------+---------------+------+
class|class_index| class_binary| count|
+-----+-----------+---------------+------+
 APO| 4.0| (12,[4],[1.0])| 10183|
 TNO| 7.0| (12,[7],[1.0])| 2728|
 AST| 10.0|(12,[10],[1.0])| 61|
 ATE| 8.0| (12,[8],[1.0])| 1380|
 IEO| 11.0|(12,[11],[1.0])| 20|
 IMB| 2.0| (12,[2],[1.0])| 15869|
 CEN| 9.0| (12,[9],[1.0])| 402|
 MBA| 0.0| (12,[0],[1.0])|669881|
 TJN| 6.0| (12,[6],[1.0])| 6531|
 AMO| 5.0| (12,[5],[1.0])| 6786|
 OMB| 1.0| (12,[1],[1.0])| 22261|
 MCA| 3.0| (12,[3],[1.0])| 14778|
 HYA| 12.0| (12,[],[])| 2|
+-----+-----------+---------------+------+

In [0]:
#useful catagorical strings have been converted, change all schema to double
df3 = encoded.withColumn("diameter",col("diameter").cast(DoubleType())).withColumn("diameter_sigma",col("diameter_sigma").cast(DoubleType())).withColumn("albedo",col("albedo").cast(DoubleType())).withColumn("epoch",col("epoch").cast(DoubleType())).withColumn("epoch_mjd",col("epoch_mjd").cast(DoubleType())).withColumn("epoch_cal",col("epoch_cal").cast(DoubleType())).withColumn("e",col("e").cast(DoubleType())).withColumn("a",col("a").cast(DoubleType())).withColumn("i",col("i").cast(DoubleType())).withColumn("q",col("q").cast(DoubleType())).withColumn("om",col("om").cast(DoubleType())).withColumn("w",col("w").cast(DoubleType())).withColumn("ma",col("ma").cast(DoubleType())).withColumn("ad",col("ad").cast(DoubleType())).withColumn("n",col("n").cast(DoubleType())).withColumn("tp",col("tp").cast(DoubleType())).withColumn("tp_cal",col("tp_cal").cast(DoubleType())).withColumn("per",col("per").cast(DoubleType())).withColumn("per_y",col("per_y").cast(DoubleType())).withColumn("moid",col("moid").cast(DoubleType())).withColumn("moid_ld",col("moid_ld").cast(DoubleType())).withColumn("sigma_e",col("sigma_e").cast(DoubleType())).withColumn("sigma_a",col("sigma_a").cast(DoubleType())).withColumn("sigma_q",col("sigma_q").cast(DoubleType())).withColumn("sigma_i",col("sigma_i").cast(DoubleType())).withColumn("sigma_om",col("sigma_om").cast(DoubleType())).withColumn("sigma_w",col("sigma_w").cast(DoubleType())).withColumn("sigma_ma",col("sigma_ma").cast(DoubleType())).withColumn("sigma_ad",col("sigma_ad").cast(DoubleType())).withColumn("sigma_n",col("sigma_n").cast(DoubleType())).withColumn("sigma_tp",col("sigma_tp").cast(DoubleType())).withColumn("sigma_per",col("sigma_per").cast(DoubleType())).withColumn("rms",col("rms").cast(DoubleType())).withColumn("class_index",col("class_index").cast(DoubleType())).withColumn("H",col("H").cast(DoubleType())).withColumn("neo",col("neo").cast(DoubleType())).withColumn("pha",col("pha").cast(DoubleType()))

In [0]:
#!!!!!UDF not working... simplify to function later
#fill in remaining null with avg of repsective column
df_stats = df3.select(_mean(col('diameter')).alias('mean')).collect()
mean_dia = df_stats[0]['mean']
df4 = df3.withColumn('diameter', when(df3.diameter.isNull(),lit(mean_dia)).otherwise(df3.diameter))
print("mean of diameter",mean_dia)

df_stats = df3.select(_mean(col('diameter_sigma')).alias('mean')).collect()
mean_diasig = df_stats[0]['mean']
df4 = df4.withColumn('diameter_sigma', when(df3.diameter_sigma.isNull(),lit(mean_diasig)).otherwise(df3.diameter_sigma))
print("mean of diameter uncertainty",mean_diasig)

df_stats = df3.select(_mean(col('albedo')).alias('mean')).collect()
mean_alb = df_stats[0]['mean']
df4 = df4.withColumn('albedo', when(df3.albedo.isNull(),lit(mean_alb)).otherwise(df3.albedo))
print("mean of albedo",mean_alb)

df_stats = df3.select(_mean(col('H')).alias('mean')).collect()
mean_H = df_stats[0]['mean']
df4 = df4.withColumn('H', when(df3.H.isNull(),lit(mean_H)).otherwise(df3.H))
print("mean of H",mean_H)

df_stats = df3.select(_mean(col('ad')).alias('mean')).collect()
mean_ad = df_stats[0]['mean']
df4 = df4.withColumn('ad', when(df3.ad.isNull(),lit(mean_ad)).otherwise(df3.ad))
print("mean of ad",mean_ad)
df_stats = df3.select(_mean(col('per')).alias('mean')).collect()
mean_per = df_stats[0]['mean']
df4 = df4.withColumn('per', when(df3.per.isNull(),lit(mean_per)).otherwise(df3.per))
print("mean of per",mean_per)
df_stats = df3.select(_mean(col('per_y')).alias('mean')).collect()
mean_per_y = df_stats[0]['mean']
df4 = df4.withColumn('per_y', when(df3.per_y.isNull(),lit(mean_per_y)).otherwise(df3.per_y))
print("mean of per_y",mean_per_y)
df_stats = df3.select(_mean(col('sigma_ad')).alias('mean')).collect()
mean_sigma_ad = df_stats[0]['mean']
df4 = df4.withColumn('sigma_ad', when(df3.sigma_ad.isNull(),lit(mean_sigma_ad)).otherwise(df3.sigma_ad))
print("mean of sigma_ad",mean_sigma_ad)
df_stats = df3.select(_mean(col('sigma_per')).alias('mean')).collect()
mean_sigma_per = df_stats[0]['mean']
df4 = df4.withColumn('sigma_per', when(df3.sigma_per.isNull(),lit(mean_sigma_per)).otherwise(df3.sigma_per))
print("mean of sigma_per",mean_sigma_per)
df_stats = df3.select(_mean(col('sigma_e')).alias('mean')).collect()
mean_sigma_e = df_stats[0]['mean']
df4 = df4.withColumn('sigma_e', when(df3.sigma_e.isNull(),lit(mean_sigma_e)).otherwise(df3.sigma_e))
print("mean of sigma_e",mean_sigma_e)
df_stats = df3.select(_mean(col('sigma_a')).alias('mean')).collect()
mean_sigma_a = df_stats[0]['mean']
df4 = df4.withColumn('sigma_a', when(df3.sigma_a.isNull(),lit(mean_sigma_a)).otherwise(df3.sigma_a))
print("mean of sigma_a",mean_sigma_a)
df_stats = df3.select(_mean(col('sigma_q')).alias('mean')).collect()
mean_sigma_q = df_stats[0]['mean']
df4 = df4.withColumn('sigma_q', when(df3.sigma_q.isNull(),lit(mean_sigma_q)).otherwise(df3.sigma_q))
print("mean of sigma_q",mean_sigma_q)
df_stats = df3.select(_mean(col('sigma_i')).alias('mean')).collect()
mean_sigma_i = df_stats[0]['mean']
df4 = df4.withColumn('sigma_i', when(df3.sigma_i.isNull(),lit(mean_sigma_i)).otherwise(df3.sigma_i))
print("mean of sigma_i",mean_sigma_i)
df_stats = df3.select(_mean(col('sigma_om')).alias('mean')).collect()
mean_sigma_om = df_stats[0]['mean']
df4 = df4.withColumn('sigma_om', when(df3.sigma_om.isNull(),lit(mean_sigma_om)).otherwise(df3.sigma_om))
print("mean of sigma_om",mean_sigma_om)
df_stats = df3.select(_mean(col('sigma_w')).alias('mean')).collect()
mean_sigma_w = df_stats[0]['mean']
df4 = df4.withColumn('sigma_w', when(df3.sigma_w.isNull(),lit(mean_sigma_w)).otherwise(df3.sigma_w))
print("mean of sigma_w",mean_sigma_w)
df_stats = df3.select(_mean(col('sigma_ma')).alias('mean')).collect()
mean_sigma_ma = df_stats[0]['mean']
df4 = df4.withColumn('sigma_ma', when(df3.sigma_ma.isNull(),lit(mean_sigma_ma)).otherwise(df3.sigma_ma))
print("mean of sigma_ma",mean_sigma_ma)
df_stats = df3.select(_mean(col('sigma_n')).alias('mean')).collect()
mean_sigma_n = df_stats[0]['mean']
df4 = df4.withColumn('sigma_n', when(df3.sigma_n.isNull(),lit(mean_sigma_n)).otherwise(df3.sigma_n))
print("mean of sigma_n",mean_sigma_n)
df_stats = df3.select(_mean(col('sigma_tp')).alias('mean')).collect()
mean_sigma_tp = df_stats[0]['mean']
df4 = df4.withColumn('sigma_tp', when(df3.sigma_tp.isNull(),lit(mean_sigma_tp)).otherwise(df3.sigma_tp))
print("mean of sigma_tp",mean_sigma_tp)
df_stats = df3.select(_mean(col('rms')).alias('mean')).collect()
mean_rms = df_stats[0]['mean']
df4 = df4.withColumn('rms', when(df3.rms.isNull(),lit(mean_rms)).otherwise(df3.rms))
print("mean of rms",mean_rms)


mean of diameter 5.50656552569257
mean of diameter uncertainty 0.47774160158044615
mean of albedo 0.1305999602098697
mean of H 16.89046394194923
mean of ad 3.4774007993846654
mean of per 5797.3555622317235
mean of per_y 15.87227335111168
mean of sigma_ad 19.130836596914445
mean of sigma_per 83261.20969836226
mean of sigma_e 0.7111202812222909
mean of sigma_a 14.013280596288977
mean of sigma_q 19.70739911736395
mean of sigma_i 1.2260299612758019
mean of sigma_om 5.378225882390413
mean of sigma_w 419083.35924330744
mean of sigma_ma 419022.78309743427
mean of sigma_n 0.05265451496723459
mean of sigma_tp 125192025.72933003
mean of rms 0.5644181450374811

In [0]:
#Additional features "diameter_est" and "psedo_target"
df4.select('diameter','albedo','H').show()

+----------------+------------------+------+
 diameter| albedo| H|
+----------------+------------------+------+
5.50656552569257|0.1305999602098697| 19.1|
5.50656552569257|0.1305999602098697|19.699|
5.50656552569257|0.1305999602098697| 16.4|
5.50656552569257|0.1305999602098697|19.048|
5.50656552569257|0.1305999602098697| 17.6|
5.50656552569257|0.1305999602098697| 16.7|
5.50656552569257|0.1305999602098697| 16.2|
 3.72| 0.032| 16.4|
5.50656552569257|0.1305999602098697| 17.3|
5.50656552569257|0.1305999602098697| 16.8|
5.50656552569257|0.1305999602098697| 16.1|
5.50656552569257|0.1305999602098697|17.891|
5.50656552569257|0.1305999602098697| 16.7|
5.50656552569257|0.1305999602098697| 16.7|
5.50656552569257|0.1305999602098697| 17.8|
5.50656552569257|0.1305999602098697| 17.6|
5.50656552569257|0.1305999602098697|18.599|
5.50656552569257|0.1305999602098697| 17.6|
5.50656552569257|0.1305999602098697| 18.0|
5.50656552569257|0.1305999602098697| 17.9|
+----------------+------------------+------+
only showing top 20 rows

In [0]:
#estimated diameter based on JPL method. link: https://cneos.jpl.nasa.gov/tools/ast_size_est.html
df5 = df4.withColumn('temp_albedo',3.1236 - (0.5 * log10(col('albedo'))))
df5 = df5.withColumn('temp_H', 0.2 * col('H'))
df5 = df5.withColumn('diameter_est', pow(10,(col('temp_albedo')-col('temp_H'))))
df5.select('diameter_est').show()

+------------------+
 diameter_est|
+------------------+
0.5567093016648205|
0.4225017164895758|
1.9303162987688838|
0.5702016604906481|
1.1107810900044575|
1.6812352133264812|
2.1165497332598933|
 3.899644439213334|
 1.275347331178873|
1.6055671640849551|
2.2162996490659483|
0.9714679510305785|
1.6812352133264812|
1.6812352133264812|
1.0130443942356375|
1.1107810900044575|
0.7011783170559964|
1.1107810900044575|
0.9239074682916438|
0.9674498858056115|
+------------------+
only showing top 20 rows

In [0]:
'''
Among NEOs, the definition of “potentially hazardous
asteroids” (PHAs) is reserved for those objects with absolute
magnitudes H  22 (i.e., diameter above ∼140 m, assuming
the average NEO albedo of 0.14 found by Mainzer et al. 2011a)
and whose minimum orbit intersection distance (MOID) is
within 0.05 au (7.5 million km) of the Earthʼs orbit.
'''

#pseudo_target based on rough classification from literature
df5 = df5.withColumn('pseudo_target', when((col('diameter_est') >=0.13) & (col('H') <=22) & (col('moid') <= 0.05), 1).otherwise(0))
distinctDF = df5.select('pseudo_target','pha').distinct()
distinctDF.show()
countDF = (df5.groupBy('pseudo_target','pha').count())
countDF.show()

+-------------+---+
pseudo_target|pha|
+-------------+---+
 1|0.0|
 0|0.0|
 1|1.0|
 0|1.0|
+-------------+---+

+-------------+---+------+
pseudo_target|pha| count|
+-------------+---+------+
 1|0.0| 31|
 0|0.0|749198|
 1|1.0| 1633|
 0|1.0| 20|
+-------------+---+------+

In [0]:
#drop unnecessary columns
df5 = df5.drop(col('equinox'))
df5 = df5.drop(col('temp_albedo'))
train_clean_df = df5.drop(col('temp_H'))

In [0]:
#FINAL DF TO BE USED
# train_clean_df.show()
train_clean_df.printSchema()

root
-- id: string (nullable = true)
-- spkid: string (nullable = true)
-- full_name: string (nullable = true)
-- pdes: string (nullable = true)
-- name: string (nullable = true)
-- prefix: string (nullable = true)
-- neo: double (nullable = true)
-- pha: double (nullable = true)
-- H: double (nullable = true)
-- diameter: double (nullable = true)
-- albedo: double (nullable = true)
-- diameter_sigma: double (nullable = true)
-- orbit_id: string (nullable = true)
-- epoch: double (nullable = true)
-- epoch_mjd: double (nullable = true)
-- epoch_cal: double (nullable = true)
-- e: double (nullable = true)
-- a: double (nullable = true)
-- q: double (nullable = true)
-- i: double (nullable = true)
-- om: double (nullable = true)
-- w: double (nullable = true)
-- ma: double (nullable = true)
-- ad: double (nullable = true)
-- n: double (nullable = true)
-- tp: double (nullable = true)
-- tp_cal: double (nullable = true)
-- per: double (nullable = true)
-- per_y: double (nullable = true)
-- moid: double (nullable = true)
-- moid_ld: double (nullable = true)
-- sigma_e: double (nullable = true)
-- sigma_a: double (nullable = true)
-- sigma_q: double (nullable = true)
-- sigma_i: double (nullable = true)
-- sigma_om: double (nullable = true)
-- sigma_w: double (nullable = true)
-- sigma_ma: double (nullable = true)
-- sigma_ad: double (nullable = true)
-- sigma_n: double (nullable = true)
-- sigma_tp: double (nullable = true)
-- sigma_per: double (nullable = true)
-- class: string (nullable = true)
-- rms: double (nullable = true)
-- class_index: double (nullable = false)
-- class_binary: vector (nullable = true)
-- diameter_est: double (nullable = true)
-- pseudo_target: integer (nullable = false)

In [0]:
# train_clean_df.write.format("orc").save("savetable/train_clean.orc")

In [0]:
train_clean_df.schema.names

Out[19]: ['id',
 'spkid',
 'full_name',
 'pdes',
 'name',
 'prefix',
 'neo',
 'pha',
 'H',
 'diameter',
 'albedo',
 'diameter_sigma',
 'orbit_id',
 'epoch',
 'epoch_mjd',
 'epoch_cal',
 'e',
 'a',
 'q',
 'i',
 'om',
 'w',
 'ma',
 'ad',
 'n',
 'tp',
 'tp_cal',
 'per',
 'per_y',
 'moid',
 'moid_ld',
 'sigma_e',
 'sigma_a',
 'sigma_q',
 'sigma_i',
 'sigma_om',
 'sigma_w',
 'sigma_ma',
 'sigma_ad',
 'sigma_n',
 'sigma_tp',
 'sigma_per',
 'class',
 'rms',
 'class_index',
 'class_binary',
 'diameter_est',
 'pseudo_target']

In [0]:
# Section 2.4 Feature Selection
# first drop all columns that have correlation 1 with some other explanatory variables
# and drop all string column
df6 = train_clean_df.drop(col("epoch_mjd"))
df6 = df6.drop(col("epoch_cal"))
df6 = df6.drop(col("moid"))
df6 = df6.drop(col("q"))
df6 = df6.drop(col("per_y"))
df6 = df6.drop(col("tp_cal"))
df6 = df6.drop(col("sigma_ad"))
df6 = df6.drop(col("sigma_ma"))
df6 = df6.drop(col("sigma_tp"))

df6 = df6.drop(col('id'))
df6 = df6.drop(col('spkid'))
df6 = df6.drop(col('orbit_id'))
df6 = df6.drop(col('full_name'))
df6 = df6.drop(col('pdes'))
df6 = df6.drop(col('name'))
df6 = df6.drop(col('prefix'))
df6 = df6.drop(col('class')) # converted into both one hot and index

In [0]:
df6.printSchema()

root
-- neo: double (nullable = true)
-- pha: double (nullable = true)
-- H: double (nullable = true)
-- diameter: double (nullable = true)
-- albedo: double (nullable = true)
-- diameter_sigma: double (nullable = true)
-- epoch: double (nullable = true)
-- e: double (nullable = true)
-- a: double (nullable = true)
-- i: double (nullable = true)
-- om: double (nullable = true)
-- w: double (nullable = true)
-- ma: double (nullable = true)
-- ad: double (nullable = true)
-- n: double (nullable = true)
-- tp: double (nullable = true)
-- per: double (nullable = true)
-- moid_ld: double (nullable = true)
-- sigma_e: double (nullable = true)
-- sigma_a: double (nullable = true)
-- sigma_q: double (nullable = true)
-- sigma_i: double (nullable = true)
-- sigma_om: double (nullable = true)
-- sigma_w: double (nullable = true)
-- sigma_n: double (nullable = true)
-- sigma_per: double (nullable = true)
-- rms: double (nullable = true)
-- class_index: double (nullable = false)
-- class_binary: vector (nullable = true)
-- diameter_est: double (nullable = true)
-- pseudo_target: integer (nullable = false)

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
colList = ["neo","H","diameter","albedo","diameter_sigma","epoch","e","a","i","om","w","ma","ad","n","tp",
           "per","moid_ld","sigma_e","sigma_a","sigma_q","sigma_i","sigma_om","sigma_w",
           "sigma_n","sigma_per","rms","diameter_est","pseudo_target","class_binary"]
assembler = VectorAssembler(inputCols=colList, outputCol="features")
df7 = assembler.transform(df6)

In [0]:
from pyspark.ml.regression import LinearRegression
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.util import MLUtils

lr = LinearRegression(featuresCol = 'features', labelCol='pha', maxIter=10, regParam=0.00001, elasticNetParam=0.2)

# Fit the model
lrModel = lr.fit(df7)

# Print the weights and intercept for linear regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [0.0035743225695748218,-4.691599564969431e-05,-6.885779205159422e-06,0.0008939581160204474,-4.868458629975019e-05,-1.4191420099668417e-08,0.0003430701664963066,-0.0,-9.766861434411865e-06,-1.1721550259058156e-07,-5.6764226513590616e-08,1.157764625911979e-07,-0.0,-0.0028583921938352195,0.0,-2.7114350115655726e-12,-4.427576017002766e-09,-0.0,-0.0,-0.0,-0.0,-3.177383468598545e-09,-0.0,-3.7861254395250526e-07,-0.0,7.310175685103568e-05,0.0,0.978903821340233,0.0003382416761600352,0.00018874838404271544,0.0010133519293710907,0.0006163474306782122,-0.0004767936564518079,-0.002735432634389802,-6.690580590638442e-05,-0.0003176284161155855,0.0015770512066259903,0.0,0.0,0.006847112132281977]
Intercept: 0.03594761687190426

In [0]:
# drop variables with coefficients 0. change
df8 = df6
for i in range(len(colList)):
  if lrModel.coefficients[i] == 0 and i <= len(colList)-2:
    df8 = df8.drop(col(colList[i]))

df8 = df8.drop(col("class_index")) # keep one hot vars
train_df_varsel = df8
train_df_varsel.printSchema()

root
-- neo: double (nullable = true)
-- pha: double (nullable = true)
-- H: double (nullable = true)
-- diameter: double (nullable = true)
-- albedo: double (nullable = true)
-- diameter_sigma: double (nullable = true)
-- epoch: double (nullable = true)
-- e: double (nullable = true)
-- i: double (nullable = true)
-- om: double (nullable = true)
-- w: double (nullable = true)
-- ma: double (nullable = true)
-- n: double (nullable = true)
-- per: double (nullable = true)
-- moid_ld: double (nullable = true)
-- sigma_om: double (nullable = true)
-- sigma_n: double (nullable = true)
-- rms: double (nullable = true)
-- class_binary: vector (nullable = true)
-- pseudo_target: integer (nullable = false)